In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
metadata_path = '../../../../../metadata/cancer_detection/matched_metadata.txt'

In [3]:
#import and format data
feature_type = 'TFBS'
metadata_columns = ['tumor_fraction','cancer_present','sample_type','Stage']
probabilities = pd.read_csv(feature_type+'_results/probabilities.txt', sep='\t')

#rename columns for each iteration
n_iter = 1+probabilities.drop(columns = ['sample','status']+metadata_columns).head().columns.astype(int).max()
probabilities[np.arange(n_iter)] = probabilities[[str(m)for m in np.arange(n_iter)]]
probabilities = probabilities.drop(columns = [str(m)for m in np.arange(n_iter)])

print(n_iter)

metadata = pd.read_csv(metadata_path, sep='\t')
metadata = metadata[['new_sample_name','file_name']].rename(columns = {'new_sample_name':'sample','file_name':'original_bam_file'})
probabilities = probabilities.merge(metadata, on = 'sample')

1000


In [4]:
probabilities['probability_cancer'] = probabilities[np.arange(n_iter)].median(axis=1)
probabilities['cancer_predicted'] = np.round(probabilities['probability_cancer'])
probabilities = probabilities.drop(columns = np.arange(n_iter))

In [5]:
export_cols = ['sample','original_bam_file','sample_type','tumor_fraction','Stage','probability_cancer','cancer_predicted']
probabilities[export_cols].to_csv('S3_cancer_detection.txt', sep='\t', index=False)